In [1]:
import requests as rq
import os
import shutil as sht
import pandas as pd
import datetime 
import warnings
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Headers

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Colllect data

In [3]:
def convert_unix_to_datetime(time):
    DT = datetime.datetime.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    return DT

In [4]:
def data_collector(start,end,stock):
    starttime = start
    endtime = end
    link = f'https://query2.finance.yahoo.com/v8/finance/chart/{stock}?period1={start}&period2={end}&interval=1m&includePrePost=true&events=div%7Csplit%7Cearn&&lang=en-US&region=US'
    print(link)
    r = rq.get(link,
           headers = headers
    )
    print(r.status_code)
    with open(f'stocks/{stock}.json','wb') as jsn:
        jsn.write(r.content)
    result = pd.read_json(f'stocks/{stock}.json')
    data = result.iloc[:1]['chart']
    timestamps = data[0][0]['timestamp']
    open_val = data[0][0]['indicators']['quote'][0]['open']
    low_val = data[0][0]['indicators']['quote'][0]['low']
    close_val = data[0][0]['indicators']['quote'][0]['close']
    high_val = data[0][0]['indicators']['quote'][0]['high']
    volume = data[0][0]['indicators']['quote'][0]['volume']
    normal_time = list(convert_unix_to_datetime(i) for i in timestamps)
    stockstill = list(stock for _ in volume)
    final = {
        'stock':stockstill,
        'timestamp':normal_time,
        'open':open_val,
        'close':close_val,
        'high':high_val,
        'low':low_val,
        'volume':volume,        
    }
    df = pd.DataFrame(final)
    return df

In [5]:
import datetime

def calculate_7_days_before(dateinput, seven_day_shift=None):
    def ensure_datetime(dateinput):
        if isinstance(dateinput, int):  
            return datetime.datetime.fromtimestamp(dateinput)
        return dateinput  

    dateinput = ensure_datetime(dateinput)

    if seven_day_shift is not None and seven_day_shift > 0:
        before_7_day_time = dateinput - datetime.timedelta(days=7)
        previous_shifts = calculate_7_days_before(before_7_day_time, seven_day_shift - 1)
        return previous_shifts + [int(dateinput.timestamp())]
    
    else:
        return [int(dateinput.timestamp())]

input_date = datetime.datetime.now()  
print(calculate_7_days_before(input_date, 3))  
print(calculate_7_days_before(
    int(datetime.datetime.now().timestamp()),
    1
))  


[1725945095, 1726549895, 1727154695, 1727759495]
[1727154695, 1727759495]


# prepare

In [6]:
os.mkdir('stocks')

In [7]:
tstlst = calculate_7_days_before(
    int(datetime.datetime.now().timestamp()),
    1
)

print("Current time (Unix):", tstlst[0])
print("7 days before time (Unix):", tstlst[1])

Current time (Unix): 1727154695
7 days before time (Unix): 1727759495


In [8]:
tstlst = calculate_7_days_before(
    int(datetime.datetime.now().timestamp()),
    1
)

df = data_collector(
    stock='TSLA',
    start=tstlst[0],  
    end=tstlst[1]  
    )

display(df)

df.to_csv('7-day-apple-per-minute-trade.csv')

https://query2.finance.yahoo.com/v8/finance/chart/TSLA?period1=1727154695&period2=1727759495&interval=1m&includePrePost=true&events=div%7Csplit%7Cearn&&lang=en-US&region=US
200


,stock,timestamp,open,close,high,low,volume
0,TSLA,2024-09-24 08:00:00,252.20,252.2400,252.4700,251.2900,0.0
1,TSLA,2024-09-24 08:01:00,252.26,252.3200,252.7000,252.2600,0.0
2,TSLA,2024-09-24 08:02:00,252.32,252.4500,252.4900,252.1900,0.0
3,TSLA,2024-09-24 08:03:00,252.44,253.0800,253.0800,252.4000,0.0
4,TSLA,2024-09-24 08:04:00,253.06,253.3700,253.3700,253.0000,0.0
...,...,...,...,...,...,...,...
4752,TSLA,2024-09-30 23:56:00,261.27,261.5000,261.5000,261.2700,0.0
4753,TSLA,2024-09-30 23:57:00,261.50,261.6014,261.6496,261.4565,0.0
4754,TSLA,2024-09-30 23:58:00,261.61,261.7025,261.7500,261.5300,0.0
4755,TSLA,2024-09-30 23:59:00,261.77,261.9900,261.9900,261.7430,0.0


In [9]:

# Create traces using go.Scatter
trace1 = go.Scatter(x=df['timestamp'], y=df['open'], mode='lines', name='Open')
trace2 = go.Scatter(x=df['timestamp'], y=df['close'], mode='lines', name='Close')
trace3 = go.Scatter(x=df['timestamp'], y=df['high'], mode='lines', name='High')
trace4 = go.Scatter(x=df['timestamp'], y=df['low'], mode='lines', name='Low')

# Create a subplot with secondary_y axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces to the figure
fig.add_trace(trace1)
fig.add_trace(trace2, secondary_y=True)
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace4, secondary_y=True)

# Update layout to show both day and time (hours and minutes)
fig.update_layout(
    height=600, 
    width=8000, 
#     title=title, 
    xaxis=dict(
        tickangle=-90,
        tickformat='%Y-%m-%d %H:%M',  # Show day and time (year-month-day hour:minute)
        dtick=3600000,  # Tick every hour (can adjust for finer control)
    ),
    template="plotly_dark"  # Apply dark theme
)

# Display the figure
fig.show()